# Naive Bayes

Naive Bayes Classifier adalah metode klasifikasi probabilistik yang didasarkan pada Teorema Bayes, yang digunakan untuk memprediksi probabilitas suatu data masuk ke dalam kelas suatu data berdasarkan fitur fiturnya, dengan asumsi bahwa setiap fitur (atribut) bersifat independen satu sama lain dan termasuk dalam keluarga algoritma **Machine learning Supervised Learning**. Biasanya algoritma ini diaplikasikan pada klasifikasi email spam atau bukan spam,prediksi cuaca, analisis sentimen teks dll.

## Langkah langkah Klasifikasi dengan menggunakan naive bayes

1. Menampilkan data
Sebelum kita melakukan klasifikasi pertama kita harus menampilkan data tersebut dan memvisualisasikan data dalam bentuk tabel

In [1]:
!pip install pymysql
!pip install pandas tabulate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.8 MB/s eta 0:00:00


In [2]:
import psycopg2
import pymysql
import pandas as pd
from tabulate import tabulate

# Koneksi ke database PostgreSQL
PG_HOST = "pg-2e640ec9-rahmatullayli123-d892.h.aivencloud.com"
PG_PORT = "21028"
PG_NAME = "defaultdb"
PG_USER = "avnadmin"
PG_PASS = "AVNS_93PD-YH-4mlx5toeFU3"

# Konfigurasi koneksi ke database MySQL
DB_HOST = "mysql-309c3d6e-mysqlirispendata7.h.aivencloud.com"
DB_PORT = 26958
DB_NAME = "iris_mysql"
DB_USER = "avnadmin"
DB_PASS = "AVNS_krNRWz9TOsAjruawhm3"

try:
    # Koneksi PostgreSQL
    conn = psycopg2.connect(
        host=PG_HOST, port=PG_PORT, dbname=PG_NAME,
        user=PG_USER, password=PG_PASS, sslmode="require"
    )
    cur = conn.cursor()

    # Ambil data dari PostgreSQL (ID, Class, Sepal Length, Sepal Width)
    cur.execute("SELECT * FROM sepal;")  # Sesuaikan tabel
    pg_rows = cur.fetchall()
    pg_columns = [desc[0] for desc in cur.description]
    pg_df = pd.DataFrame(pg_rows, columns=pg_columns)

    # Koneksi MySQL
    mysql_conn = pymysql.connect(
        host=DB_HOST, port=DB_PORT, user=DB_USER,
        password=DB_PASS, database=DB_NAME,
        cursorclass=pymysql.cursors.DictCursor
    )
    mysql_cur = mysql_conn.cursor()

    # Ambil data dari MySQL (ID, Petal Length, Petal Width)
    mysql_cur.execute("SELECT * FROM petal;")  # Sesuaikan tabel
    mysql_rows = mysql_cur.fetchall()
    mysql_df = pd.DataFrame(mysql_rows)

    # Menggabungkan kedua tabel berdasarkan ID
    combined_df = pd.merge(pg_df, mysql_df, on=["id","class"], how="inner")  # Inner join

    # Mengatur urutan kolom agar class berada di sebelah ID
    ordered_columns = ["id", "class", "sepal_length", "sepal_width", "petal_length", "petal_width"]
    combined_df = combined_df[ordered_columns]

    # Mengurutkan data berdasarkan ID
    combined_df = combined_df.sort_values(by="id").reset_index(drop=True)

    # Menampilkan 3 data awal, tanda "..." di tengah, dan 3 data akhir
    df_selected = pd.concat([
        combined_df.head(15),
        pd.DataFrame([["..."] * len(combined_df.columns)], columns=combined_df.columns),
        combined_df.tail(15)
    ])
    # Menampilkan hasil dalam format tabel
    print(tabulate(df_selected, headers="keys", tablefmt="psql", showindex=False))

except Exception as e:
    print("Error:", e)

finally:
    # Menutup koneksi
    if cur:
        cur.close()
    if conn:
        conn.close()

+------+----------------+----------------+---------------+----------------+---------------+
| id   | class          | sepal_length   | sepal_width   | petal_length   | petal_width   |
|------+----------------+----------------+---------------+----------------+---------------|
| 1    | Iris-setosa    | 20.1           | 30.5          | 80.4           | 70.0          |
| 2    | Iris-setosa    | 4.9            | 3.0           | 1.4            | 0.2           |
| 3    | Iris-setosa    | 4.7            | 3.2           | 1.3            | 0.2           |
| 4    | Iris-setosa    | 4.6            | 3.1           | 1.5            | 0.2           |
| 5    | Iris-setosa    | 5.0            | 3.6           | 1.4            | 0.2           |
| 6    | Iris-setosa    | 5.4            | 3.9           | 1.7            | 0.4           |
| 7    | Iris-setosa    | 4.6            | 3.4           | 1.4            | 0.3           |
| 8    | Iris-setosa    | 5.0            | 3.4           | 1.5            | 0.2 

2. Melakukan Pembersihan data

**Pembersihan Data** adalah proses membersihkan dan mempersiapkan data sebelum analisis atau pemodelan. Ini mencakup:  

1. **Menghapus Data Duplikat** – Menghilangkan data yang berulang untuk menghindari bias.  
2. **Menangani Data Kosong (Missing Values)** – Mengisi dengan rata-rata, median, atau menghapusnya jika terlalu banyak.  
3. **Menghapus Outlier** – Menggunakan metode statistik (z-score, IQR, LOF) untuk mendeteksi data ekstrem.  
4. **Standarisasi Format Data** – Menyamakan format tanggal, teks, dan angka agar konsisten.  
5. **Menghapus Data Tidak Relevan** – Menghapus kolom atau baris yang tidak diperlukan.  

Pada tahap ini kita akan mencari oulier dengan menggunakan LOF

In [4]:
import numpy as np
import pandas as pd
import psycopg2
import pymysql
from sklearn.neighbors import LocalOutlierFactor
from tabulate import tabulate

# Koneksi ke database PostgreSQL
PG_HOST = "pg-2e640ec9-rahmatullayli123-d892.h.aivencloud.com"
PG_PORT = "21028"
PG_NAME = "defaultdb"
PG_USER = "avnadmin"
PG_PASS = "AVNS_93PD-YH-4mlx5toeFU3"

# Konfigurasi koneksi ke database MySQL
DB_HOST = "mysql-309c3d6e-mysqlirispendata7.h.aivencloud.com"
DB_PORT = 26958
DB_NAME = "iris_mysql"
DB_USER = "avnadmin"
DB_PASS = "AVNS_krNRWz9TOsAjruawhm3"

try:
    # Koneksi PostgreSQL
    conn = psycopg2.connect(
        host=PG_HOST, port=PG_PORT, dbname=PG_NAME,
        user=PG_USER, password=PG_PASS, sslmode="require"
    )
    cur = conn.cursor()

    # Ambil data dari PostgreSQL
    cur.execute("SELECT * FROM sepal;")
    pg_rows = cur.fetchall()
    pg_columns = [desc[0] for desc in cur.description]
    pg_df = pd.DataFrame(pg_rows, columns=pg_columns)

    # Koneksi MySQL
    mysql_conn = pymysql.connect(
        host=DB_HOST, port=DB_PORT, user=DB_USER,
        password=DB_PASS, database=DB_NAME,
        cursorclass=pymysql.cursors.DictCursor
    )
    mysql_cur = mysql_conn.cursor()

    # Ambil data dari MySQL
    mysql_cur.execute("SELECT * FROM petal;")
    mysql_rows = mysql_cur.fetchall()
    mysql_df = pd.DataFrame(mysql_rows)

    # Normalisasi nama kolom agar seragam
    pg_df.columns = pg_df.columns.str.strip().str.lower()
    mysql_df.columns = mysql_df.columns.str.strip().str.lower()

    # Jika ada duplikasi kolom 'class', hapus salah satunya sebelum merge
    if "class" in pg_df.columns and "class" in mysql_df.columns:
        mysql_df = mysql_df.drop(columns=["class"])

    # Menggabungkan kedua tabel berdasarkan ID
    combined_df = pd.merge(pg_df, mysql_df, on="id", how="inner")

    # Pilih hanya kolom numerik
    numeric_cols = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
    X = combined_df[numeric_cols].astype(float)

    # Inisialisasi model LOF
    clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
    y_pred = clf.fit_predict(X)

    # Tambahkan kolom outlier ke DataFrame (True jika -1, False jika 1)
    combined_df["outlier"] = y_pred == -1

    # Urutkan berdasarkan ID
    combined_df = combined_df.sort_values(by="id").reset_index(drop=True)

    # Menampilkan seluruh data dalam format tabel
    print(tabulate(combined_df, headers="keys", tablefmt="psql", showindex=False))

except Exception as e:
    print("Error:", e)

finally:
    # Menutup koneksi database
    if cur: cur.close()
    if conn: conn.close()
    if mysql_cur: mysql_cur.close()
    if mysql_conn: mysql_conn.close()


+------+-----------------+----------------+---------------+----------------+---------------+-----------+
|   id | class           |   sepal_length |   sepal_width |   petal_length |   petal_width | outlier   |
|------+-----------------+----------------+---------------+----------------+---------------+-----------|
|    1 | Iris-setosa     |           20.1 |          30.5 |           80.4 |          70   | True      |
|    2 | Iris-setosa     |            4.9 |           3   |            1.4 |           0.2 | False     |
|    3 | Iris-setosa     |            4.7 |           3.2 |            1.3 |           0.2 | False     |
|    4 | Iris-setosa     |            4.6 |           3.1 |            1.5 |           0.2 | False     |
|    5 | Iris-setosa     |            5   |           3.6 |            1.4 |           0.2 | False     |
|    6 | Iris-setosa     |            5.4 |           3.9 |            1.7 |           0.4 | False     |
|    7 | Iris-setosa     |            4.6 |           3

3. Menormalisasi data outlier

Normalisasi data adalah proses menyesuaikan skala nilai fitur sehingga berada dalam rentang tertentu, biasanya 0 hingga 1 atau -1 hingga 1. Tujuannya adalah:

✅ Mengurangi pengaruh outlier ekstrem

✅ Membantu model machine learning bekerja lebih baik

✅ Menyamakan skala antara fitur dengan unit berbeda

### **🔹 Apa yang Bisa Dilakukan pada Outlier?**  

Setelah mendeteksi outlier, kita bisa memilih beberapa tindakan, tergantung pada kasusnya:  

1️⃣ **Menghapus Outlier** 🚮  
   - Jika outlier adalah **kesalahan data atau noise**, sebaiknya dihapus.  
   - Contoh: Kesalahan input manusia, data yang tidak relevan.  

2️⃣ **Mengubah atau Menyesuaikan Nilai** ✏️  
   - Menggunakan **winsorizing** (mengganti outlier dengan nilai batas tertentu).  
   - Contoh: Jika nilai sangat tinggi, bisa diubah ke persentil 95%.  

3️⃣ **Menggunakan Transformasi Data** 🔄  
   - Metode seperti **log transform, square root, atau Box-Cox** bisa mengurangi efek outlier.  
   - Cocok untuk data dengan distribusi skewed.  

4️⃣ **Menormalisasi atau Standarisasi** 📏  
   - **Min-Max Scaling** atau **Z-Score Standardization** membuat data lebih seimbang.  
   - Berguna jika outlier tidak ingin dihapus tetapi skalanya perlu disesuaikan.  

5️⃣ **Menggunakan Model yang Tahan Outlier** 🏗️  
   - Beberapa model seperti **Decision Tree atau Random Forest** tidak terlalu terpengaruh oleh outlier.  
   - Bisa dipilih jika outlier tidak bisa dihilangkan.  

🔥 **Kesimpulan:**  
✅ Jika outlier adalah **kesalahan**, hapus.  
✅ Jika outlier **bermakna**, normalisasi atau gunakan model yang lebih tahan.  
✅ Pilih metode yang sesuai dengan **konteks data dan tujuan analisis.** 🚀

Pada tahap ini kita akan menghapus outlier dan menampilkan tabelnya tanpa outlier

In [5]:
# Hapus data yang terdeteksi sebagai outlier
cleaned_df = combined_df[combined_df["outlier"] == False]

# Reset index setelah penghapusan
cleaned_df = cleaned_df.reset_index(drop=True)

# Tampilkan data setelah outlier dihapus
print("Data setelah outlier dihapus:")
print(tabulate(cleaned_df, headers="keys", tablefmt="psql", showindex=False))


Data setelah outlier dihapus:
+------+-----------------+----------------+---------------+----------------+---------------+-----------+
|   id | class           |   sepal_length |   sepal_width |   petal_length |   petal_width | outlier   |
|------+-----------------+----------------+---------------+----------------+---------------+-----------|
|    2 | Iris-setosa     |            4.9 |           3   |            1.4 |           0.2 | False     |
|    3 | Iris-setosa     |            4.7 |           3.2 |            1.3 |           0.2 | False     |
|    4 | Iris-setosa     |            4.6 |           3.1 |            1.5 |           0.2 | False     |
|    5 | Iris-setosa     |            5   |           3.6 |            1.4 |           0.2 | False     |
|    6 | Iris-setosa     |            5.4 |           3.9 |            1.7 |           0.4 | False     |
|    7 | Iris-setosa     |            4.6 |           3.4 |            1.4 |           0.3 | False     |
|    8 | Iris-setosa     

4. Mengklasifikasikan dengan menggunakan teorema bayes


Teorema Bayes dapat dinyatakan sebagai berikut:
$$
P(Y|X) = \frac{P(X|Y) \cdot P(Y)}{P(X)}
$$


**Penjelasan:**  
- \( P(Y|X) \): Probabilitas posterior (kelas \( Y \) diberikan fitur \( X \)).  
- \( P(X|Y) \): Likelihood (probabilitas fitur \( X \) diberikan kelas \( Y \)).  
- \( P(Y) \): Prior probability (probabilitas awal kelas \( Y \)).  
- \( P(X) \): Evidence (probabilitas fitur \( X \) secara keseluruhan).  

**Tujuan:**  
Mencari kelas \( Y \) dengan nilai \( P(Y|X) \) tertinggi.


Asumsi "Naive"
- setiap fitur besifat independen tidak ada kaitannya fitur satu dengan fitur yang lainnya
- yang artinya Probabilitas suatu fitur tidak dipengaruhi oleh fitur lainnya

Langkah langkah penerapan teorema bayes pada dataset
1. Hitung probabilitas awal \( P(Y) \):
- hitung frekuensi setiap kelas dalam dataset
2. Hitung likelihood \( P(X|Y) \):
- untuk setiap fitur hitung probabilitas kemunculannya dalam setiap kelas
3. Gabungkan dengan Teorema Bayes

Dengan rumus:$$
P(Y|X) = \frac{P(X|Y) \cdot P(Y)}{P(X)}
$$
4. prediksi kelas:
- prediksi kelas berdasarkan dengan nilai tertinggi \( P(Y|X) \)

In [6]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from tabulate import tabulate

# Pilih fitur numerik untuk model Naïve Bayes
numeric_cols = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
X_full = combined_df[numeric_cols]
y_full = combined_df["class"]

# Inisialisasi model Naïve Bayes
nb_full = GaussianNB()

# Latih model dengan data **tanpa menghapus outlier**
nb_full.fit(X_full, y_full)

# Hitung likelihood P(X|Y) untuk setiap sampel
likelihoods_full = nb_full.predict_proba(X_full)

# Konversi likelihood menjadi DataFrame untuk setiap kelas
likelihood_df_full = pd.DataFrame(likelihoods_full, columns=nb_full.classes_)

# Gabungkan hasil likelihood dengan dataset utama, hindari duplikasi kolom
combined_df = pd.concat([combined_df.drop(columns=list(nb_full.classes_), errors='ignore'), likelihood_df_full], axis=1)

# Tambahkan hasil akhir prediksi ke dataset
combined_df["final_prediction"] = nb_full.predict(X_full)

# Pastikan kolom "class" berada setelah "id"
ordered_columns = ["id", "class", "sepal_length", "sepal_width", "petal_length", "petal_width"] + list(nb_full.classes_) + ["final_prediction"]

# Pilih hanya kolom yang ada di dataset untuk menghindari error
combined_df = combined_df[[col for col in ordered_columns if col in combined_df.columns]]

# Tampilkan hasil akhir dalam bentuk tabel
print("Hasil Naïve Bayes **tanpa menghapus outlier**:")
print(tabulate(combined_df, headers="keys", tablefmt="psql", showindex=False))


Hasil Naïve Bayes **tanpa menghapus outlier**:
+------+-----------------+----------------+---------------+----------------+---------------+---------------+-------------------+------------------+--------------------+
|   id | class           |   sepal_length |   sepal_width |   petal_length |   petal_width |   Iris-setosa |   Iris-versicolor |   Iris-virginica | final_prediction   |
|------+-----------------+----------------+---------------+----------------+---------------+---------------+-------------------+------------------+--------------------|
|    1 | Iris-setosa     |           20.1 |          30.5 |           80.4 |          70   |   1           |       0           |      0           | Iris-setosa        |
|    2 | Iris-setosa     |            4.9 |           3   |            1.4 |           0.2 |   1           |       2.58588e-12 |      4.00852e-20 | Iris-setosa        |
|    3 | Iris-setosa     |            4.7 |           3.2 |            1.3 |           0.2 |   1           |

In [7]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from tabulate import tabulate

# Pilih fitur numerik untuk model Naïve Bayes
numeric_cols = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
X_full = combined_df[numeric_cols]
y_full = combined_df["class"]

# Inisialisasi model Naïve Bayes
nb_full = GaussianNB()

# Latih model dengan data **tanpa menghapus outlier**
nb_full.fit(X_full, y_full)

# Hitung likelihood P(X|Y) untuk setiap sampel
likelihoods_full = nb_full.predict_proba(X_full)

# Konversi likelihood menjadi DataFrame untuk setiap kelas
likelihood_df_full = pd.DataFrame(likelihoods_full, columns=nb_full.classes_)

# Gabungkan hasil likelihood dengan dataset utama
combined_df = pd.concat([combined_df, likelihood_df_full], axis=1)

# Tambahkan hasil akhir prediksi ke dataset
combined_df["final_prediction"] = nb_full.predict(X_full)

# Hapus kolom class_frequency agar sesuai dengan dataset tanpa outlier
if "class_frequency" in combined_df.columns:
    combined_df = combined_df.drop(columns=["class_frequency"])

# Urutkan ulang kolom agar sesuai dengan versi tanpa outlier
ordered_columns = ["id", "sepal_length", "sepal_width", "petal_length", "petal_width",
                   "class"] + list(nb_full.classes_) + ["final_prediction"]

# Pastikan hanya kolom yang ada di dataset yang dipilih
combined_df = combined_df[[col for col in ordered_columns if col in combined_df.columns]]

# Tampilkan hasil akhir dalam bentuk tabel
print("Hasil Naïve Bayes **tanpa menghapus outlier**:")
print(tabulate(combined_df, headers="keys", tablefmt="psql", showindex=False))


Hasil Naïve Bayes **tanpa menghapus outlier**:
+------+----------------+---------------+----------------+---------------+-----------------+---------------+---------------+-------------------+-------------------+------------------+------------------+--------------------+
|   id |   sepal_length |   sepal_width |   petal_length |   petal_width | class           |   Iris-setosa |   Iris-setosa |   Iris-versicolor |   Iris-versicolor |   Iris-virginica |   Iris-virginica | final_prediction   |
|------+----------------+---------------+----------------+---------------+-----------------+---------------+---------------+-------------------+-------------------+------------------+------------------+--------------------|
|    1 |           20.1 |          30.5 |           80.4 |          70   | Iris-setosa     |   1           |   1           |       0           |       0           |      0           |      0           | Iris-setosa        |
|    2 |            4.9 |           3   |            1.4 

5. Hitung akurasi

Untuk menghitung akurasi kita bisa membandingkan label kelas prediksi dengan label kelas sebenarnya dengan menggunakan naive bayes

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Pisahkan fitur (X) dan label kelas (y)
X = cleaned_df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
y = cleaned_df["class"]

# Bagi dataset menjadi data latih (80%) dan data uji (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model Naïve Bayes
model = GaussianNB()

# Latih model dengan data latih
model.fit(X_train, y_train)

# Prediksi data uji
y_pred = model.predict(X_test)

# Hitung akurasi
accuracy = accuracy_score(y_test, y_pred)

# Tampilkan hasil akurasi
print(f"Akurasi model setelah penghapusan outlier: {accuracy:.2%}")


Akurasi model setelah penghapusan outlier: 100.00%


In [9]:
from sklearn.metrics import accuracy_score

# Hitung akurasi model pada seluruh data tanpa menghapus outlier
accuracy_full = accuracy_score(combined_df["class"], combined_df["final_prediction"])

# Tampilkan hasil akurasi dalam bentuk persentase
print(f"Akurasi model tanpa menghapus outlier: {accuracy_full:.2%}")


Akurasi model tanpa menghapus outlier: 96.00%


## Kesimpulan


Berdasarkan hasil perhitungan akurasi menggunakan **Naïve Bayes**, diperoleh dua nilai akurasi:  

1. **Setelah menghapus outlier** → Akurasi **lebih tinggi**, menunjukkan bahwa model bekerja lebih baik dengan data yang bersih dan tidak mengandung anomali.  
2. **Tanpa menghapus outlier** → Akurasi **lebih rendah**, karena adanya data outlier yang dapat menyebabkan model salah dalam mengklasifikasikan kelas.  

Dari hasil ini, dapat disimpulkan bahwa **penghapusan outlier dapat meningkatkan akurasi model Naïve Bayes**, karena model menjadi lebih fokus pada pola utama dalam data tanpa terganggu oleh nilai ekstrem. 🚀